In [11]:
using DataStructures
using FileIO
using Meshes
using MeshIO
using Makie
using GLMakie
using GeometryBasics
using ProgressBars
using JLD2
using Serialization

In [ ]:
include("contractions.jl")
include("visualisation.jl")
include("visualisation.jl")

visualize (generic function with 3 methods)

In [ ]:
cubedata = loadsc_OLD("other_ply/cube.ply")
cube = initialContractedSimplicialComplex2D(cubedata)
visualize(cube)

GLMakie.Screen(...)

In [61]:
bunidata = deserialize("bunidata.dat")
buni = initialContractedSimplicialComplex2D(bunidata)

ContractedSimplicialComplex2D(SimplicialComplex2D(Vector{Float32}[[-0.09231, 0.13236, 0.01822], [-0.09218, 0.13235, 0.01722], [-0.09277, 0.13099, 0.01723], [-0.08717, 0.11039, 0.01836], [-0.08896, 0.11186, 0.01834], [-0.08744, 0.11045, 0.01734], [-0.08784, 0.10502, 0.01437], [-0.08724, 0.10636, 0.01436], [-0.08725, 0.10635, 0.01336], [-0.08785, 0.10502, 0.01337]  …  [-0.0419, 0.03367, 0.02694], [-0.0428, 0.03443, 0.02962], [-0.04345, 0.03445, 0.03123], [-0.04408, 0.03448, 0.03283], [-0.04464, 0.03465, 0.03432], [-0.04532, 0.03459, 0.03598], [-0.04592, 0.03469, 0.03752], [-0.04664, 0.03455, 0.03923], [-0.04724, 0.03464, 0.04077], [-0.04787, 0.03465, 0.04236]], Set([17984, 19700, 29965, 11950, 21664, 30270, 1703, 12427, 7685, 18374  …  4724, 25754, 4496, 10808, 8546, 4159, 24824, 8992, 15380, 31794]), Dict{Int64, Set{Tuple{Int64, Int64}}}(17984 => Set([(16945, 17984), (17762, 17984), (17984, 22404), (17984, 24281), (16944, 17984), (8914, 17984)]), 19700 => Set([(19700, 19730), (19662, 19

In [69]:
fillholes!(buni.contracted)
getholes(buni.contracted)

Set{Vector{Tuple{Int64, Int64}}}()

In [70]:
getholes(buni.contracted)

Set{Vector{Tuple{Int64, Int64}}}()

In [97]:
getholes(buni.original)

Set{Vector{Tuple{Int64, Int64}}} with 5 elements:
  [(31923, 32324), (32324, 32600), (16987, 32600), (252, 16987), (251, 252), (2…
  [(34641, 34677), (34677, 34712), (34712, 34747), (34747, 34782), (34782, 3481…
  [(15450, 32692), (31957, 32692), (31932, 31957), (17194, 31932), (17194, 1728…
  [(20907, 21550), (21550, 32827), (28144, 32827), (22960, 28144), (17196, 2296…
  [(4268, 19808), (16419, 19808), (16363, 16419), (16141, 16363), (16140, 16141…

In [71]:
visualize(buni)

GLMakie.Screen(...)

In [76]:
marked_vertices = Set{Int}()
for edge in getborder(buni.original)
    push!(marked_vertices, edge[1])
    push!(marked_vertices, edge[2])
end
marks::Vector{GeometryBasics.Point{3,Float32}} = [GeometryBasics.Point{3,Float32}(buni.original.coords[x]) for x in marked_vertices]
visualize(buni, marks)

GLMakie.Screen(...)

In [7]:
pq = PriorityQueue()
for (edge, triangle) in ProgressBar(buni.contracted._edge_to_triangles)
    pq[edge] = error(buni, edge)
end

0.0%┣                                   ┫ 0/208.4k [00:14<-813:-51:-37, -14s/it]
0.0%┣                                       ┫ 1/208.4k [00:25<Inf:Inf, InfGs/it]
0.0%┣                                       ┫ 11/208.4k [00:25<145:35:05, 3s/it]
1.8%┣▊                                      ┫ 3.7k/208.4k [00:25<22:59, 148it/s]
3.5%┣█▍                                     ┫ 7.3k/208.4k [00:25<11:37, 289it/s]
5.2%┣██                                    ┫ 10.9k/208.4k [00:25<07:38, 432it/s]
7.1%┣██▊                                   ┫ 14.7k/208.4k [00:25<05:34, 580it/s]
8.8%┣███▍                                  ┫ 18.4k/208.4k [00:25<04:23, 723it/s]
10.5%┣███▉                                 ┫ 21.8k/208.4k [00:25<03:38, 855it/s]
12.5%┣████▌                               ┫ 26.0k/208.4k [00:26<02:59, 1.0kit/s]
14.3%┣█████▏                              ┫ 29.8k/208.4k [00:26<02:33, 1.2kit/s]
16.0%┣█████▊                              ┫ 33.4k/208.4k [00:26<02:14, 1.3kit/s]
17.8%┣██████▍               

In [8]:
for i in ProgressBar(1:3)
    first = dequeue!(pq)
    while !issafe(buni.contracted, first)
        first = dequeue!(pq)
    end

    new_vertex = contract!(buni, first)
    for edge in buni.contracted._vertex_to_edges[new_vertex]
        pq[edge] = error(buni, edge)
        for triangle in buni.contracted._edge_to_triangles[edge]
            pq[triangle] = error(buni, triangle)
        end
    end
end

0.0%┣                                                ┫ 0/3 [00:00<00:00, -0s/it]


BoundsError: BoundsError: attempt to access Set{Int64} with 1 element at index [2]